In [1]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import pandas as pd
from openpyxl import load_workbook
import re
import io
import os
import pdb

import sys
sys.path.append(os.path.abspath('../kev/'))

import eq_data
import eq_preproc
import eq_evaluator
import eq_postproc
import eq_writer


# basic input ------------------------------------------------
 
def cl_scripts_load(_sep, _subdir, _file):
    
    # if specific file selected it should be XLSX one
    if _file != "":
        
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        _file = _subdir + _file
        
        # open excel file
        with open(_file, "rb") as f:
            inmemory_file = io.BytesIO(f.read())
        wb = load_workbook(inmemory_file, read_only = True)
                
        r = re.compile(r'heats')
        heats_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], index_col = 0)
        
        r = re.compile(r'targets')
        targets = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = None)
        
        r = re.compile(r'enthalpies')
        delta_H = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
    # use a bunch of plain text files instead
    else:
          
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        file_names = list(os.listdir(path = _subdir))

        r = re.compile(r'heats')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        heats_data = pd.read_csv(file, sep = _sep, decimal = ",", index_col = 0)
        
        r = re.compile(r'targets')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        targets = pd.read_csv(file, sep = _sep, header = None)
        
        r = re.compile(r'enthalpies')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        delta_H = pd.read_csv(file, sep = _sep)
        
    return heats_data, targets, delta_H

In [2]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np
import re

# basic preprocessing ----------------------------------------
    
def cl_preproc(con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type):
    
    # checking if there are several series
    
    if 'series' not in con_data.columns:        
        con_data['series'] = ''
    
    if 'series' not in heats_data.index:
        heats_data.loc['series'] = '' # probably not needed
    
    if 'volumes' not in heats_data.index:
        heats_data.loc['volumes'] = 1        

    # series variables
    
    ser_info = con_data['series'].to_numpy()
    ser_unique = np.unique(ser_info)
    ser_num = np.shape(np.unique(ser_info))[0]
    
    ser_counts = con_data.groupby(['series']).size().tolist();
    
    # reading volumes from experimental data
    volumes = heats_data.loc['volumes'].to_numpy()
    
    # reading exp heats from experimental data
    heats = heats_data.loc['observation'].to_numpy() - heats_data.loc['dilution'].to_numpy()
        
    # transforming deviation to the matrix form
    devs = heats_data.loc['deviation'].to_numpy()
    devs = np.diag((1 / (np.array(devs) * np.array(devs))) * (np.sum(np.array(devs) * np.array(devs)) / len(devs)))
    
    # creating vector of known enthalpies
    dH_par = np.hstack((np.zeros(len(prod_names_con)), np.transpose(delta_H.drop('Reaction', axis = 1).to_numpy())[0]))
    
    # number of constant to find
    tar_names = set(targets.to_numpy()[0][1:])
    tar_num = [index for index, item in enumerate(prod_names) if item in tar_names]
    
    # number of enthalpy to find
    dH_names = np.hstack((prod_names_con, np.transpose(delta_H.drop('Value', axis = 1).to_numpy())[0]))
    un_el = set(prod_names) - set(dH_names)
    dH_ind_wtf = list([index for index, item in enumerate(prod_names) if item in un_el])
    un_el = set(prod_names) - (set(prod_names) - set(dH_names))
    dH_ind = list([index for index, item in enumerate(prod_names) if item in un_el])
    
    # creating array of diff total concentrations
    con_matrix_red = []
    for s in range(ser_num):
        for k in range(ser_counts[s] - 1):
            con_matrix_red.append(con_matrix[int(np.sum(ser_counts[:s])) + k + 1])
            
    if _calor_setup_type == 'DSC' or 'overfilled':
        _calor_setup_type_coeff = [[1] for i in range(len(con_matrix_red))]
    
    if _calor_setup_type == 'ampoule':
        _calor_setup_type_coeff = 1 / np.array(con_matrix_red)[:, idx] 
    
    return (ser_num, ser_counts, ser_info, volumes, heats, devs, tar_num, dH_ind, dH_ind_wtf, dH_par, 
            _calor_setup_type_coeff)

In [3]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np

def cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type):

    # finding the increment in equilibrium concentrations of products - needed for dH calculation
    vol_active = 0.988 # terrible solution. Gotta introduce new variable for operating volume 
                       # of calorimetric cell for overfilled mode
    dif_conc = []        
             
    for s in range(ser_num):    
        for k in range(ser_counts[s] - 1):
            
            if _calor_setup_type == 'overfilled':
                factor = 1 - (volumes[int(np.sum(ser_counts[:s])) + k - s] / vol_active)
            else:
                factor = 1
            
            dif_conc.append(c_res_out[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out[int(np.sum(ser_counts[:s])) +
                                                                                       k] * factor)
            
    dif_conc, devs, volumes = np.array(dif_conc).astype(np.float32), devs.astype(np.float32), volumes.astype(np.float32)
    
    # finding vector of experimental values
    s = [0] * len(heats)
    q_corr = deepcopy(heats)
    for i in range(len(heats)):
        for j in range(len(volumes[dH_ind])):
            
            if _calor_setup_type == 'overfilled':
                q_corr[i] -= vol_active * dif_conc[i, dH_ind[j]] * dH_par[j]
            else:
                q_corr[i] -= volumes[dH_ind[j]] * dif_conc[i, dH_ind[j]] * dH_par[j]
                
    if _calor_setup_type == 'overfilled':
        A = (vol_active * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff)
    else:
        A = (np.array(volumes).reshape(len(volumes), 1) * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff) 
    
    AT = np.transpose(A)
    LTP = np.linalg.inv(np.dot(np.dot(AT, devs), A))
    RTP = np.dot(np.dot(AT, devs), q_corr)
    dH_res = np.dot(LTP, RTP)
    dH_res_tmp = dH_par

    for i in range(len(dH_ind_wtf)):
        dH_res_tmp = np.insert(dH_res_tmp, dH_ind_wtf[i], dH_res[i])

    dH_res_out = dH_res_tmp

    F_min, heats_calc = 0, []
    
    for i in range(len(heats)): 
        
        if _calor_setup_type == 'overfilled':
            F_min += (vol_active * np.sum(dH_res_out * dif_conc[i] * np.array(_calor_setup_type_coeff[i])) - heats[i]) ** 2
            heats_calc.append(np.sum(dH_res_out * dif_conc[i] * vol_active * _calor_setup_type_coeff[i]))
        
        else:
            F_min += (np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i])) - heats[i]) ** 2
            heats_calc.append(np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i])))
    print('\neq con')
    print(np.array(c_res_out))
    print('\ndif con')
    print(dif_conc)
    print('\nvol')
    print(volumes)
    print('\ndH par')
    print(dH_par)
    print('\nsum of terms')
    print(np.array(heats_calc))
    print(heats)
    
    #print(sum(heats))
    #print(dH_res_out)
    #print(np.sum(dif_conc[i]))
    #print(np.sum(volumes[i]))
    #print(np.sum(np.array(_calor_setup_type_coeff[i])))
    
    # calculating st. dev. of dH_res / checking the quality of linear regression
    # consider making it in separate function
    
    disp = (F_min / (len(heats) - len(dH_ind_wtf)) * (np.diag(LTP))) ** 0.5  
    
    return F_min, dH_res_out, disp

In [4]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

from copy import deepcopy
import eq_evaluator
# import also script cl_enthalpies when needed

# main Hooke-Jeeves procedure --------------------------------
def cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type):

    basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type)
    print(new_min)

    for i in range(max_iter):
        print('\niteration', i, lg_k[tar_num])
        if F_min <= new_min:
        
            if step_init < HJ_eps:
                print("p1 " + str(step_init) + " " + str(HJ_eps))
                break
            
            else: 
                 
                step_init /= 2
            
                basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
                print('\nF min more ', new_min, F_min, lg_k[tar_num])
                print(str(step_init))
        else:
            F_min = deepcopy(new_min)
            print('\nF min less', new_min, F_min, lg_k[tar_num])
            print(str(step_init))
            basis_point_n = deepcopy(lg_k)
            lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])

            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                    con_matrix, ign_indices)

            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
            print('\nF min less after', new_min, F_min, lg_k[tar_num])
        
            if F_min <= new_min:
                lg_k = deepcopy(basis_point_n)
                print('\nconstant less after', basis_point[tar_num], lg_k[tar_num]) 
            else:
                F_min = new_min

            basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
        
            if F_min <= new_min:
            
                print('\nF min p2', new_min, F_min, lg_k[tar_num])
                if step_init < HJ_eps:
                    print("p2")
                    break
            
                else: 
                
                    step_init /= 2
        
            else:

                F_min = deepcopy(new_min)
            
                print('\nF min p3.1', new_min, F_min, lg_k[tar_num])
            
                basis_point_n = deepcopy(lg_k)
                lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])
            
                c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                        con_matrix, ign_indices)
    
                new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                              , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
        
                if F_min <= new_min:
                    lg_k = deepcopy(basis_point_n)
            
                print('\nF min p3.2', new_min, F_min, lg_k[tar_num])
    
    return F_min, lg_k, dH_res_out, c_res_out, disp

# exploratory move function used in main function ------------------------
def cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type):

    basis_point = deepcopy(lg_k)
    
    c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                con_matrix, ign_indices)

    new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)

    #if F_min > new_min:
    F_min = new_min
    
    for j in range(len(tar_num)): 
        #pdb.set_trace()
        print('input', lg_k[tar_num[j]], F_min)
        
        lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
        
        c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)
        #print('\na1 ', new_min, F_min)
        new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                   _calor_setup_type_coeff, _calor_setup_type)
        print('right', lg_k[tar_num[j]], new_min, F_min)
        #print('\na2 ', new_min, F_min)
        if F_min <= new_min:
            
            lg_k[tar_num[j]] = lg_k[tar_num[j]] - 2 * step_init
            
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                        con_matrix, ign_indices)
        
            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                                  , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
            print('left', lg_k[tar_num[j]], new_min, F_min)
            #print('\na3 ', new_min, F_min)
            if F_min <= new_min:
                
                lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
                new_min = deepcopy(F_min)
                #print('\na4 ', new_min, F_min)
            #else:
                
                #new_min = deepcopy(F_min)
                
        #else:
            
            #new_min = deepcopy(F_min)
    print('return', basis_point[tar_num[j]], lg_k[tar_num[j]])
    #print('\na5 ', new_min, F_min)
    return basis_point, lg_k, new_min, c_res_out, disp

In [5]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------
import numpy as np
import eq_evaluator
# import also script cl_enthalpies when needed

def cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                      ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                      _calor_setup_type_coeff, _calor_setup_type):
    
    # creating constants vectors ---------------------------------
    lg_k_1, lg_k_2 = np.array(lg_k), np.array(lg_k)
    heats_one, heats_two = [0] * len(heats), [0] * len(heats)
    jc_mtrx = [[0] * len(heats) for i in range(len(tar_num))]

    for i in range(len(tar_num)):
        # calculating the heats for Jacobi matrix computing
        lg_k_1[tar_num[i]] += HJ_eps
        lg_k_2[tar_num[i]] -= HJ_eps
    
        # if lg_k is overestimated
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_1,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])
    
        for o in range(len(heats)):
            heats_one[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
        # if lg_k is underestimated    
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_2,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])    
   
        for o in range(len(heats)):
            heats_two[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
    # calculating the Jacobi matrix    
    jc_mtrx[i] = (np.array(heats_one) - np.array(heats_two)) / (2 * HJ_eps)

    jc_mtrx, devs = np.array(jc_mtrx).astype(np.float32), devs.astype(np.float32)

    cov_matrix = (F_min / (len(heats) - len(tar_num))) * np.linalg.inv(np.dot(np.dot(jc_mtrx, devs), np.transpose(jc_mtrx)))

    errs_values = np.diag(cov_matrix) ** 0.5
    return errs_values

In [6]:
# input for xlsx file
_subdir = "calorimetry"
_sep = ";"
_file = "test_5.xlsx"
_calor_setup_type = 'DSC' # use 'DSC' for test_1 and test_2; use 'ampoule' for test_3; use 'overfilled' for test_4

max_iter, eps, HJ_eps, step_init = 100, 0.0000001, 0.0001, 0.5

# load data

st_coeff_data, lg_k_data, con_data, type_con, component_name_for_yields = eq_data.eq_scripts_load(_sep, _subdir, _file)
heats_data, targets, delta_H = cl_scripts_load(_sep, _subdir, _file)

# preproc

st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, idx, type_con = eq_preproc.eq_preproc(
    st_coeff_data, con_data, type_con, lg_k_data, component_name_for_yields)

(ser_num, ser_counts, ser_info, volumes, heats, devs, 
 tar_num, dH_ind, dH_ind_wtf, dH_par, _calor_setup_type_coeff) = cl_preproc(
    con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type)
#----
#lg_k[tar_num] = 10 # it is just for quick variation of initial approximation of the constant to be optimized. Delete in the end
#----

# calculation of eq concentration (starting point)
c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)

# calculation of F_min and unknown enthalpies (starting point)
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type)

# F_min assignment for further calculations
new_min = F_min

# constant refinment
if len(tar_num) > 0:
    F_min, lg_k, dH_res_out, c_res_out, disp = cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type)


F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type)

# error estimation
#if len(tar_num) > 0:
    #errs_values = cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                                    #ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                                    #_calor_setup_type_coeff, _calor_setup_type)


eq con
[[5.00000006e-17 1.00000000e-03 5.00000006e-17 2.50000006e-31]
 [8.60140605e-05 9.18640463e-04 7.90159963e-05 6.79648669e-07]
 [1.77254089e-04 8.44358803e-04 1.49666050e-04 2.65289194e-06]
 [2.73467516e-04 7.76789118e-04 2.12426591e-04 5.80917722e-06]
 [3.74349759e-04 7.15502450e-04 2.67848170e-04 1.00268898e-05]
 [4.79561977e-04 6.60030596e-04 3.16525578e-04 1.51793632e-05]
 [5.88748060e-04 6.09888268e-04 3.59070534e-04 2.11402080e-05]
 [7.01548795e-04 5.64591699e-04 3.96088626e-04 2.77875498e-05]
 [8.17613281e-04 5.23673006e-04 4.28162005e-04 3.50070941e-05]
 [9.36606978e-04 4.86690193e-04 4.55837431e-04 4.26940518e-05]
 [1.05821690e-03 4.53233305e-04 4.79619144e-04 5.07541084e-05]
 [1.18215443e-03 4.22927479e-04 4.99965593e-04 5.91036541e-05]
 [1.30815631e-03 3.95433646e-04 5.17289021e-04 6.76694899e-05]
 [1.43598439e-03 3.70447624e-04 5.31957004e-04 7.63881953e-05]
 [1.56542446e-03 3.47698171e-04 5.44295222e-04 8.52053055e-05]
 [1.69628469e-03 3.26944465e-04 5.54590892e-04 

 [2.00144028e-03 8.53773947e-05 5.40362914e-04 3.42001627e-04]]

dif con
[[ 4.32341294e-05 -1.23180173e-04  1.19877339e-04  1.63894299e-06]
 [ 5.00872993e-05 -1.12389229e-04  1.05712461e-04  5.01839895e-06]
 [ 5.75231061e-05 -1.00977355e-04  9.08855727e-05  8.43892303e-06]
 [ 6.53160314e-05 -8.93412289e-05  7.59644972e-05  1.17293448e-05]
 [ 7.31902765e-05 -7.79295733e-05  6.15708996e-05  1.47167257e-05]
 [ 8.08689365e-05 -6.71596354e-05  4.82624273e-05  1.72606833e-05]
 [ 8.81214219e-05 -5.73455145e-05  3.64366751e-05  1.92777061e-05]
 [ 9.47915105e-05 -4.86642130e-05  2.62914418e-05  2.07470002e-05]
 [ 1.00800986e-04 -4.11627552e-05  1.78425962e-05  2.16997287e-05]
 [ 1.06135769e-04 -3.47909809e-05  1.09757530e-05  2.22001127e-05]
 [ 1.10825233e-04 -2.94408128e-05  5.50413370e-06  2.23268562e-05]
 [ 1.14922885e-04 -2.49792847e-05  1.21535493e-06  2.21593873e-05]
 [ 1.18492062e-04 -2.12709601e-05 -2.09818722e-06  2.17697743e-05]
 [ 1.21596931e-04 -1.81904306e-05 -4.62297248e-06  2.121


eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 6.19747302e-32]
 [1.74806360e-05 8.49622079e-04 1.48519343e-04 1.94688477e-07]
 [4.09819167e-05 7.06323328e-04 2.89464838e-04 8.89586295e-07]
 [7.32340964e-05 5.73051260e-04 4.19668912e-04 2.30473038e-06]
 [1.17887729e-04 4.53742295e-04 5.34906487e-04 4.72875121e-06]
 [1.78858991e-04 3.52607122e-04 6.30669539e-04 8.45887546e-06]
 [2.58620749e-04 2.72277205e-04 7.04165348e-04 1.36564562e-05]
 [3.56586358e-04 2.12062256e-04 7.56185075e-04 2.02205436e-05]
 [4.69504619e-04 1.68393066e-04 7.90613224e-04 2.78358151e-05]
 [5.93354489e-04 1.36882735e-04 8.12199855e-04 3.61390840e-05]
 [7.24731880e-04 1.13827668e-04 8.24945397e-04 4.48334923e-05]
 [8.61194604e-04 9.65761343e-05 8.31708457e-04 5.37121353e-05]
 [1.00110048e-03 8.33486039e-05 8.34403277e-04 6.26402774e-05]
 [1.14336114e-03 7.29683909e-05 8.34292228e-04 7.15322073e-05]
 [1.28724856e-03 6.46506372e-05 8.32214395e-04 8.03336755e-05]
 [1.43226621e-03 5.78619686e-05 8.28737427e-04 


eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 7.15673020e-32]
 [1.74738972e-05 8.49645308e-04 1.48466148e-04 2.24655583e-07]
 [4.09432711e-05 7.06420579e-04 2.89231693e-04 1.02548288e-06]
 [7.31065439e-05 5.73272196e-04 4.19099490e-04 2.65322075e-06]
 [1.17553984e-04 4.54114062e-04 5.33829171e-04 5.43425049e-06]
 [1.78114538e-04 3.53104469e-04 6.28930393e-04 9.70067492e-06]
 [2.57189480e-04 2.72816533e-04 7.01655424e-04 1.56270531e-05]
 [3.54198611e-04 2.12545072e-04 7.52831696e-04 2.30911068e-05]
 [4.65972498e-04 1.68755803e-04 7.86355629e-04 3.17306731e-05]
 [5.88585150e-04 1.37105801e-04 8.06984385e-04 4.11314890e-05]
 [7.18700747e-04 1.13918444e-04 8.18732710e-04 5.09554027e-05]
 [8.53916212e-04 9.65524889e-05 8.24477355e-04 6.09668829e-05]
 [9.92609365e-04 8.32300496e-05 8.26149267e-04 7.10128414e-05]
 [1.13370126e-03 7.27726034e-05 8.25023925e-04 8.09962988e-05]
 [1.27646779e-03 6.43925261e-05 8.21949857e-04 9.08563291e-05]
 [1.42041350e-03 5.75537453e-05 8.17501166e-04 


eq con
[[9.39265577e-18 1.00000000e-03 9.06073454e-17 7.63970894e-32]
 [1.80304098e-05 8.50216519e-04 1.47880236e-04 2.39354024e-07]
 [4.21676904e-05 7.07709235e-04 2.87878796e-04 1.08971892e-06]
 [7.50893713e-05 5.75411346e-04 4.16804011e-04 2.80954089e-06]
 [1.20295720e-04 4.57150279e-04 5.30498504e-04 5.72874298e-06]
 [1.81436240e-04 3.56899551e-04 6.24661931e-04 1.01740549e-05]
 [2.60723462e-04 2.77031007e-04 6.96760457e-04 1.63075459e-05]
 [3.57532329e-04 2.16784825e-04 7.47685908e-04 2.39971416e-05]
 [4.68812689e-04 1.72742815e-04 7.81221796e-04 3.28774942e-05]
 [5.90795452e-04 1.40717156e-04 8.01971977e-04 4.25325417e-05]
 [7.20251975e-04 1.17134721e-04 8.13851385e-04 5.26204509e-05]
 [8.54831822e-04 9.94021700e-05 8.19693604e-04 6.29009533e-05]
 [9.92932631e-04 8.57570305e-05 8.21418571e-04 7.32165567e-05]
 [1.13347974e-03 7.50215943e-05 8.20304421e-04 8.34668100e-05]
 [1.27574702e-03 6.64037124e-05 8.17206710e-04 9.35882848e-05]
 [1.41923489e-03 5.93614851e-05 8.12707069e-04 

...


eq con
[[8.91133675e-18 1.00000000e-03 9.10886656e-17 7.37325320e-32]
 [1.71385933e-05 8.49316977e-04 1.48787506e-04 2.31629351e-07]
 [4.01913200e-05 7.05701567e-04 2.89917770e-04 1.05842249e-06]
 [7.18511855e-05 5.72105941e-04 4.20176617e-04 2.74231773e-06]
 [1.15730307e-04 4.52483044e-04 5.35267529e-04 5.62690984e-06]
 [1.75722023e-04 3.51076603e-04 6.30593612e-04 1.00653230e-05]
 [2.54308984e-04 2.70555221e-04 7.03297553e-04 1.62462362e-05]
 [3.50944175e-04 2.10241369e-04 7.54184667e-04 2.40418393e-05]
 [4.62426994e-04 1.66546706e-04 7.87228320e-04 3.30670796e-05]
 [5.84783746e-04 1.35054817e-04 8.07284950e-04 4.28819082e-05]
 [7.14648541e-04 1.12039669e-04 8.18438055e-04 5.31288333e-05]
 [8.49607002e-04 9.48367340e-05 8.23599656e-04 6.35603372e-05]
 [9.88035740e-04 8.16601209e-05 8.24715501e-04 7.40165375e-05]
 [1.12885839e-03 7.13302275e-05 8.23065803e-04 8.43967991e-05]
 [1.27135421e-03 6.30609623e-05 8.19499406e-04 9.46383504e-05]
 [1.41503081e-03 5.63185084e-05 8.14588968e-04 


eq con
[[8.91133675e-18 1.00000000e-03 9.10886656e-17 7.38569842e-32]
 [1.71385075e-05 8.49317280e-04 1.48786814e-04 2.32018075e-07]
 [4.01908273e-05 7.05702837e-04 2.89914738e-04 1.06018490e-06]
 [7.18495561e-05 5.72108830e-04 4.20169210e-04 2.74683574e-06]
 [1.15726033e-04 4.52487912e-04 5.35253519e-04 5.63605174e-06]
 [1.75712463e-04 3.51083122e-04 6.30571013e-04 1.00814023e-05]
 [2.54290565e-04 2.70562291e-04 7.03264993e-04 1.62717260e-05]
 [3.50913418e-04 2.10247694e-04 7.54141258e-04 2.40789226e-05]
 [4.62381503e-04 1.66551456e-04 7.87173328e-04 3.31173210e-05]
 [5.84722378e-04 1.35057744e-04 8.07217727e-04 4.29462038e-05]
 [7.14571043e-04 1.12040879e-04 8.18358137e-04 5.32075415e-05]
 [8.49513626e-04 9.48364664e-05 8.23506815e-04 6.36534462e-05]
 [9.87926992e-04 8.16586350e-05 8.24609724e-04 7.41238004e-05]
 [1.12873489e-03 7.13277563e-05 8.22947247e-04 8.45178267e-05]
 [1.27121663e-03 6.30577011e-05 8.19368347e-04 9.47726713e-05]
 [1.41487982e-03 5.63146165e-05 8.14445762e-04 


eq con
[[8.91133675e-18 1.00000000e-03 9.10886656e-17 7.38985149e-32]
 [1.71384789e-05 8.49317381e-04 1.48786583e-04 2.32147793e-07]
 [4.01906629e-05 7.05703260e-04 2.89913726e-04 1.06077301e-06]
 [7.18490124e-05 5.72109794e-04 4.20166739e-04 2.74834335e-06]
 [1.15724607e-04 4.52489537e-04 5.35248845e-04 5.63910222e-06]
 [1.75709273e-04 3.51085297e-04 6.30563473e-04 1.00867674e-05]
 [2.54284420e-04 2.70564650e-04 7.03254129e-04 1.62802308e-05]
 [3.50903156e-04 2.10249805e-04 7.54126774e-04 2.40912952e-05]
 [4.62366326e-04 1.66553041e-04 7.87154981e-04 3.31340834e-05]
 [5.84701904e-04 1.35058721e-04 8.07195300e-04 4.29676545e-05]
 [7.14545188e-04 1.12041283e-04 8.18331475e-04 5.32338001e-05]
 [8.49482474e-04 9.48363771e-05 8.23475842e-04 6.36845085e-05]
 [9.87890712e-04 8.16581393e-05 8.24574436e-04 7.41595839e-05]
 [1.12869369e-03 7.13269318e-05 8.22907697e-04 8.45582014e-05]
 [1.27117073e-03 6.30566131e-05 8.19324626e-04 9.48174799e-05]
 [1.41482945e-03 5.63133181e-05 8.14397990e-04 


eq con
[[8.90905545e-18 1.00000000e-03 9.10909469e-17 7.39022165e-32]
 [1.71342333e-05 8.49313150e-04 1.48790800e-04 2.32162110e-07]
 [4.01811922e-05 7.05693872e-04 2.89923033e-04 1.06085523e-06]
 [7.18333401e-05 5.72094393e-04 4.20181867e-04 2.74861526e-06]
 [1.15702176e-04 4.52467820e-04 5.35269848e-04 5.63981547e-06]
 [1.75680567e-04 3.51058186e-04 6.30588989e-04 1.00883628e-05]
 [2.54251144e-04 2.70534448e-04 7.03281257e-04 1.62833045e-05]
 [3.50867432e-04 2.10219214e-04 7.54152233e-04 2.40964278e-05]
 [4.62329671e-04 1.66523977e-04 7.87176454e-04 3.31416744e-05]
 [5.84664983e-04 1.35032041e-04 8.07211739e-04 4.29778949e-05]
 [7.14508114e-04 1.12017141e-04 8.18342685e-04 5.32467317e-05]
 [8.49445123e-04 9.48146062e-05 8.23482032e-04 6.37000892e-05]
 [9.87852894e-04 8.16384667e-05 8.24575963e-04 7.41777291e-05]
 [1.12865523e-03 7.13090777e-05 8.22904945e-04 8.45788072e-05]
 [1.27113149e-03 6.30403250e-05 8.19317959e-04 9.48404350e-05]
 [1.41478932e-03 5.62983800e-05 8.14387737e-04 


eq con
[[8.90905545e-18 1.00000000e-03 9.10909469e-17 7.39437727e-32]
 [1.71342047e-05 8.49313251e-04 1.48790569e-04 2.32291908e-07]
 [4.01810277e-05 7.05694296e-04 2.89922020e-04 1.06144371e-06]
 [7.18327961e-05 5.72095358e-04 4.20179394e-04 2.75012383e-06]
 [1.15700750e-04 4.52469445e-04 5.35265170e-04 5.64286791e-06]
 [1.75677375e-04 3.51060363e-04 6.30581443e-04 1.00937315e-05]
 [2.54244994e-04 2.70536809e-04 7.03270386e-04 1.62918149e-05]
 [3.50857164e-04 2.10221326e-04 7.54137740e-04 2.41088086e-05]
 [4.62314484e-04 1.66525563e-04 7.87158094e-04 3.31584478e-05]
 [5.84644496e-04 1.35033018e-04 8.07189297e-04 4.29993595e-05]
 [7.14482243e-04 1.12017545e-04 8.18316005e-04 5.32730071e-05]
 [8.49413952e-04 9.48145171e-05 8.23451039e-04 6.37311710e-05]
 [9.87816593e-04 8.16379708e-05 8.24540654e-04 7.42135347e-05]
 [1.12861401e-03 7.13082529e-05 8.22865371e-04 8.46192064e-05]
 [1.27108556e-03 6.30392365e-05 8.19274213e-04 9.48852701e-05]
 [1.41473892e-03 5.62970810e-05 8.14339937e-04 

In [7]:
print(np.array(c_res_out))
#print(c_res_out[int(np.sum(ser_counts[:0])) + 1])

[[8.90677467e-18 1.00000000e-03 9.10932277e-17 7.39266936e-32]
 [1.71299743e-05 8.49308970e-04 1.48794901e-04 2.32241317e-07]
 [4.01716410e-05 7.05684697e-04 2.89931831e-04 1.06123166e-06]
 [7.18173984e-05 5.72079478e-04 4.20195757e-04 2.74964140e-06]
 [1.15679036e-04 4.52446919e-04 5.35288510e-04 5.64205491e-06]
 [1.75650268e-04 3.51032166e-04 6.30610728e-04 1.00926427e-05]
 [2.54214796e-04 2.70505430e-04 7.03302946e-04 1.62906340e-05]
 [3.50826577e-04 2.10189683e-04 7.54170440e-04 2.41077522e-05]
 [4.62285424e-04 1.66495710e-04 7.87188741e-04 3.31576544e-05]
 [5.84617820e-04 1.35005853e-04 8.07216951e-04 4.29988706e-05]
 [7.14458106e-04 1.11993206e-04 8.18340547e-04 5.32728050e-05]
 [8.49392185e-04 9.47927950e-05 8.23472717e-04 6.37312156e-05]
 [9.87796925e-04 8.16185503e-05 8.24559826e-04 7.42137826e-05]
 [1.12859616e-03 7.12908151e-05 8.22882396e-04 8.46196188e-05]
 [1.27106928e-03 6.30234964e-05 8.19289409e-04 9.48858144e-05]
 [1.41472399e-03 5.62827959e-05 8.14353573e-04 1.049734

In [8]:
#print('\nStoich coeff data')
#print(st_coeff_data)

#print('\nlg K')
#print(lg_k_data)

#print('\nconcentrations')
#print(con_data)

#print('\ntype con')
#print(type_con)

#print('\nexperimental data')
#print(heats_data)

print('\n_calor_setup_type_coeff')
print(_calor_setup_type_coeff)

print('\ntargets')
print(targets)

#print('\ndelta_H')
#print(delta_H)


_calor_setup_type_coeff
[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

targets
            0   1    2
0  constants   HL  H2L


In [9]:
print('\nSeries')
print(ser_num)
print(ser_counts)
print(ser_info)

print('\nSt coeff matr')
print(st_coeff_matrix)

print('\nprod names')
print(prod_names)
print(prod_names_con)

print('\nconst')
print(lg_k)

print('\nconcentrations')
print(con_matrix)

print(ign_indices)

print('\ntype con')
print(type_con)

print('\nvolumes')
print(volumes)

print('\nheats')
print(heats)

print('\ndeviations')
print(devs)

print('\nwhat to find and what is known')
print(tar_num)
print(dH_ind)
print(dH_ind_wtf)

print('\nenthalpies')
print(dH_par)


Series
1
[21]
['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']

St coeff matr
[[1 0]
 [0 1]
 [1 1]
 [2 1]]

prod names
['H', 'L', 'HL', 'H2L']
['H', 'L']

const
[[0.        ]
 [0.        ]
 [4.00970459]
 [5.96929932]]

concentrations
[[1.00000000e-16 1.00000000e-03]
 [1.66389351e-04 9.98336106e-04]
 [3.32225914e-04 9.96677741e-04]
 [4.97512438e-04 9.95024876e-04]
 [6.62251656e-04 9.93377483e-04]
 [8.26446281e-04 9.91735537e-04]
 [9.90099010e-04 9.90099010e-04]
 [1.15321252e-03 9.88467875e-04]
 [1.31578947e-03 9.86842105e-04]
 [1.47783251e-03 9.85221675e-04]
 [1.63934426e-03 9.83606557e-04]
 [1.80032733e-03 9.81996727e-04]
 [1.96078431e-03 9.80392157e-04]
 [2.12071778e-03 9.78792822e-04]
 [2.28013029e-03 9.77198697e-04]
 [2.43902439e-03 9.75609756e-04]
 [2.59740260e-03 9.74025974e-04]
 [2.75526742e-03 9.72447326e-04]
 [2.91262136e-03 9.70873786e-04]
 [3.06946688e-03 9.69305331e-04]
 [3.22580645e-03 9.67741935e-04]]
[]

type con
0    tot
1    tot
Name: 0, dtype: object



In [10]:
print('\nEq conc')
print(np.array(c_res_out))
print(np.shape(c_res_out))


Eq conc
[[8.90677467e-18 1.00000000e-03 9.10932277e-17 7.39266936e-32]
 [1.71299743e-05 8.49308970e-04 1.48794901e-04 2.32241317e-07]
 [4.01716410e-05 7.05684697e-04 2.89931831e-04 1.06123166e-06]
 [7.18173984e-05 5.72079478e-04 4.20195757e-04 2.74964140e-06]
 [1.15679036e-04 4.52446919e-04 5.35288510e-04 5.64205491e-06]
 [1.75650268e-04 3.51032166e-04 6.30610728e-04 1.00926427e-05]
 [2.54214796e-04 2.70505430e-04 7.03302946e-04 1.62906340e-05]
 [3.50826577e-04 2.10189683e-04 7.54170440e-04 2.41077522e-05]
 [4.62285424e-04 1.66495710e-04 7.87188741e-04 3.31576544e-05]
 [5.84617820e-04 1.35005853e-04 8.07216951e-04 4.29988706e-05]
 [7.14458106e-04 1.11993206e-04 8.18340547e-04 5.32728050e-05]
 [8.49392185e-04 9.47927950e-05 8.23472717e-04 6.37312156e-05]
 [9.87796925e-04 8.16185503e-05 8.24559826e-04 7.42137826e-05]
 [1.12859616e-03 7.12908151e-05 8.22882396e-04 8.46196188e-05]
 [1.27106928e-03 6.30234964e-05 8.19289409e-04 9.48858144e-05]
 [1.41472399e-03 5.62827959e-05 8.14353573e-04

In [11]:
print('\nlg K')
print(lg_k)

print('\nF_min')
print(F_min)

print('\nFinal enthalpies')
print(dH_res_out)

print('\ncalculated error of enthalpy')
print(disp)

#if len(tar_num) > 0:
    #print('\ncalculated error of constant')
    #print(errs_values)


lg K
[[0.        ]
 [0.        ]
 [4.00970459]
 [5.96929932]]

F_min
1.5911821031978063e-09

Final enthalpies
[0.0 0.0 9.966471594968002 33.40691817323163]

calculated error of enthalpy
[0.00210566 0.01673268]


In [12]:
#import matplotlib.pyplot

#s = 4.5

#xxx = []
#FFF = []
#while s < 5.5:
    #lg_k[tar_num] = s
    
    #c_res_out = eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
        #con_matrix, ign_indices, ser_counts, ser_info)
        
    #F_min, dH_res_out = enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff)
    
    #xxx.append(s)
    #FFF.append(F_min)
    #s += 0.02


#matplotlib.pyplot.plot(xxx, FFF, 'ro-')

#import sys
#import os
##sys.path.append('../kev_calorim')
#print(os.path.abspath('../kev/'))
#print(os.getcwd())

#lg_k is array

In [13]:
import matplotlib.pyplot as plt
from copy import copy, deepcopy

def risovalka(max_iter, eps, st_coeff_matrix, type_con, lg_k, con_matrix, ign_indices, ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type):

    s = [[0] * 42 for i in range(42)]
    const_x = [0] * 42
    const_y = [0] * 42
    s = np.array(s)

    i, a = 3, 0

    for counter_x in range(len(s)):
        j = 5
        const_x[counter_x] = round(i, 2)
        for counter_y in range(len(s)):
            lg_k[2], lg_k[3] = np.array(i), np.array(j)
            const_y[counter_y] = round(j, 2)
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                            con_matrix, ign_indices)
        
            F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
            
            s[counter_x, counter_y] = float(F_min * 100000000)
        
            j += 0.05
    
        i += 0.05

    s = pd.DataFrame(data = s, index = const_y, columns = const_x)

    fig, ax = plt.subplots(figsize=(20, 20))
    im = ax.imshow(s, cmap = 'RdYlGn_r')
    ax.set_xticks(np.arange(len(const_x)))
    ax.set_yticks(np.arange(len(const_y)))

    ax.set_xlabel('lg K1')
    ax.set_ylabel('lg K2')

    ax.set_xticklabels(const_x)
    ax.set_yticklabels(const_y)

    plt.show()
    return s

In [14]:
#comment this section for reducing the waiting time
#s = risovalka(max_iter, eps, st_coeff_matrix, type_con, lg_k, con_matrix, ign_indices, ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff, _calor_setup_type)
#with pd.ExcelWriter('d:/KEVreplica/output/calorimetry/test_5.xlsx', mode = "w") as output:
    #s.to_excel(output)

In [15]:
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type) 


eq con
[[8.90677467e-18 1.00000000e-03 9.10932277e-17 7.39266936e-32]
 [1.71299743e-05 8.49308970e-04 1.48794901e-04 2.32241317e-07]
 [4.01716410e-05 7.05684697e-04 2.89931831e-04 1.06123166e-06]
 [7.18173984e-05 5.72079478e-04 4.20195757e-04 2.74964140e-06]
 [1.15679036e-04 4.52446919e-04 5.35288510e-04 5.64205491e-06]
 [1.75650268e-04 3.51032166e-04 6.30610728e-04 1.00926427e-05]
 [2.54214796e-04 2.70505430e-04 7.03302946e-04 1.62906340e-05]
 [3.50826577e-04 2.10189683e-04 7.54170440e-04 2.41077522e-05]
 [4.62285424e-04 1.66495710e-04 7.87188741e-04 3.31576544e-05]
 [5.84617820e-04 1.35005853e-04 8.07216951e-04 4.29988706e-05]
 [7.14458106e-04 1.11993206e-04 8.18340547e-04 5.32728050e-05]
 [8.49392185e-04 9.47927950e-05 8.23472717e-04 6.37312156e-05]
 [9.87796925e-04 8.16185503e-05 8.24559826e-04 7.42137826e-05]
 [1.12859616e-03 7.12908151e-05 8.22882396e-04 8.46196188e-05]
 [1.27106928e-03 6.30234964e-05 8.19289409e-04 9.48858144e-05]
 [1.41472399e-03 5.62827959e-05 8.14353573e-04 